In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from dateutil import tz
import math
from sklearn import preprocessing

In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_cars_spec = pd.read_csv('/content/drive/My Drive/Visualization/fullspecs.csv', index_col=0, header=None).T

In [0]:
df_cars_spec.head(1)

In [0]:
df_cars_spec.rename(columns={ df_cars_spec.columns[1]: "MRSP" }, inplace = True)

In [0]:
df_cars_spec.rename(columns={ df_cars_spec.columns[0]: "Car" }, inplace = True)

In [0]:
pd.options.display.max_rows = 300

In [0]:
round(df_cars_spec.isnull().mean()*100,2).sort_values()

In [0]:
df_cleaned_car_spec = df_cars_spec.loc[:, df_cars_spec.isna().mean() < .5]

In [41]:
df_cleaned_car_spec.shape

(32316, 105)

In [0]:
round(df_cleaned_car_spec.isnull().mean()*100,2).sort_values()

In [0]:
df_cleaned_car_spec = df_cleaned_car_spec.dropna(thresh=30)

In [0]:
df_cleaned_car_spec.head(10)

In [77]:
df_cleaned_car_spec['Gas Mileage'].head(5)

1    22 mpg City/28 mpg Hwy
2    22 mpg City/28 mpg Hwy
3    22 mpg City/27 mpg Hwy
4    22 mpg City/28 mpg Hwy
5    21 mpg City/27 mpg Hwy
Name: Gas Mileage, dtype: object

In [0]:
df_cleaned_car_spec[['City Mileage','Highway Mileage']] = df_cleaned_car_spec['Gas Mileage'].str.split("/",expand=True,)

In [0]:
df_cleaned_car_spec['Highway Mileage'] = df_cleaned_car_spec['Highway Mileage'].apply(lambda x: float(x.split(' ')[0]))

In [0]:
df_cleaned_car_spec['Highway Mileage'].value_counts()

In [0]:
df_cleaned_car_spec = df_cleaned_car_spec.dropna(axis=0, subset=['SAE Net Torque @ RPM'])

In [0]:
del df_cleaned_car_spec['Car']

In [0]:
df_cleaned_car_spec['Passenger Doors'] = df_cleaned_car_spec['Passenger Doors'].astype(int)

In [0]:

df_cleaned_car_spec= df_cleaned_car_spec[(df_cleaned_car_spec['Engine Displacement']!="6.2l")]

In [0]:
df_cleaned_car_spec['Engine Displacement'] = df_cleaned_car_spec['Engine Displacement'].str.replace(' ', '')
df_cleaned_car_spec['Engine Displacement'] = df_cleaned_car_spec['Engine Displacement'].str.replace(''', '')

In [0]:
df_cleaned_car_spec['Engine Displacement'] = df_cleaned_car_spec['Engine Displacement'].astype(float)

In [0]:
df_cleaned_car_spec[['Engine Fuel Type','Engine Displacement']] = df_cleaned_car_spec['Engine'].str.split(",",expand=True,)

In [0]:
df_cleaned_car_spec['Drivetrain'] = df_cleaned_car_spec['Drivetrain'].apply(lambda x: x.lower())

In [0]:
df_cleaned_car_spec['Drivetrain'] = df_cleaned_car_spec['Drivetrain'].str.replace('-', ' ')

In [0]:
wheelDriveMapping = {'front wheel drive' : 0, 'rear wheel drive' : 1, 'all wheel drive' : 2,'4 wheel drive' : 2,'four wheel drive' : 2, 'awd': 2,
'2 wheel drive' : 0, '4wd' : 2, '2wd' : 0,'rwd' : 1}
df_cleaned_car_spec['Drivetrain'] = df_cleaned_car_spec['Drivetrain'].apply(lambda x: wheelDriveMapping[x])

In [0]:
import re
transmission = []    
for values in df_cleaned_car_spec['Transmission']:
  res = re.search("[0-9]-[sS]peed ", values)
  if res is None:
    transmission.append('5-Speed ')
  if res is not None:
    transmission.append(res.group())

In [227]:
df_cleaned_car_spec['SAE Net Torque @ RPM'].isna().value_counts()

False    24256
True         2
Name: SAE Net Torque @ RPM, dtype: int64

In [0]:
df_cleaned_car_spec['Gears'] = df_cleaned_car_spec['Gears'].apply(lambda x: int(x.split('-')[0]))

In [0]:
df_cleaned_car_spec['Transmission'] = df_cleaned_car_spec['Transmission'].apply(lambda x: 1 if ("automatic" in str(x)) else 0)

In [194]:
df_cleaned_car_spec.head(1)

,Car,MRSP,Drivetrain,Passenger Capacity,Passenger Doors,Body Style,Transmission,Base Curb Weight (lbs),Front Hip Room (in),Front Leg Room (in),Second Shoulder Room (in),Passenger Volume (ft³),Second Head Room (in),Front Shoulder Room (in),Second Hip Room (in),Front Head Room (in),Second Leg Room (in),Wheelbase (in),Min Ground Clearance (in),"Track Width, Front (in)","Width, Max w/o mirrors (in)","Track Width, Rear (in)","Height, Overall (in)","Fuel Tank Capacity, Approx (gal)",Fuel Economy Est-Combined (MPG),EPA Fuel Economy Est - City (MPG),EPA Fuel Economy Est - Hwy (MPG),SAE Net Torque @ RPM,Fuel System,Engine Type,SAE Net Horsepower @ RPM,Displacement,First Gear Ratio (:1),Sixth Gear Ratio (:1),Trans Description Cont.,Fourth Gear Ratio (:1),Second Gear Ratio (:1),Reverse Ratio (:1),Fifth Gear Ratio (:1),Trans Type,Third Gear Ratio (:1),Final Drive Axle Ratio (:1),Brake Type,Rear Brake Rotor Diam x Thickness (in),Disc - Rear (Yes or ),Brake ABS System,Front Brake Rotor Diam x Thickness (in),Disc - Front (Yes or ),Steering Type,Turning Diameter - Curb to Curb (ft),Spare Tire Size,Front Tire Size,Rear Tire Size,Front Wheel Size (in),Spare Wheel Material,Front Wheel Material,Rear Wheel Size (in),Rear Wheel Material,Spare Wheel Size (in),Suspension Type - Front,Suspension Type - Rear,Air Bag-Frontal-Driver,Air Bag-Frontal-Passenger,Air Bag-Passenger Switch (On/Off),Air Bag-Side Body-Front,Air Bag-Side Body-Rear,Air Bag-Side Head-Front,Air Bag-Side Head-Rear,Brakes-ABS,Child Safety Rear Door Locks,Daytime Running Lights,Traction Control,Night Vision,Rollover Protection Bars,Fog Lamps,Parking Aid,Tire Pressure Monitor,Back-Up Camera,Stability Control,Other Features,Basic Miles/km,Basic Years,Corrosion Miles/km,Corrosion Years,Drivetrain Miles/km,Drivetrain Years,Roadside Assistance Miles/km,Roadside Assistance Years,Maximum Alternator Capacity (amps),Cold Cranking Amps @ 0° F (Primary),Wt Distributing Hitch - Max Tongue Wt. (lbs),Dead Weight Hitch - Max Tongue Wt. (lbs),Wt Distributing Hitch - Max Trailer Wt. (lbs),Dead Weight Hitch - Max Trailer Wt. (lbs),"Length, Overall (in)",Turning Diameter - Wall to Wall (ft),Shock Absorber Diameter - Front (mm),Shock Absorber Diameter - Rear (mm),Stabilizer Bar Diameter - Front (in),Total Cooling System Capacity (qts),City Mileage,Highway Mileage,Engine Fuel Type,Engine Displacement,Gears
1,2019 Acura RDX Specs: FWD w/Technology Pkg,40600.0,0,5,4,Sport Utility,0,3790.0,55.0,41.6,56.6,104.0,38.3,59.7,49.9,39.6,38.4,108.3,5.7,64.2,74.8,64.7,65.7,17.1,24,22,28,280 @ 1600,Gasoline Direct Injection,Turbo Premium Unleaded I-4,272 @ 6500,2.0 L/122,5.25,1.00,Automatic w/OD,1.60,3.27,3.97,1.30,10,2.19,4.17,4-Wheel Disc,12.2,Yes,4-Wheel,12.4,Yes,Rack-Pinion,39,NaN,P235/55HR19,P235/55HR19,19 X 8,NaN,Aluminum,19 X 8,Aluminum,NaN,Strut,Multi-Link,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,Yes,Yes,Yes,Vehicle Stability Assist Electronic Stability ...,"50,000",4,Unlimited,5,"70,000",6,"50,000",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,28.0,Turbo Premium Unleaded I-4,2.0,0


In [0]:
df_cleaned_car_spec['EPA Classification'] = df_cleaned_car_spec['EPA Classification'].str.replace(' ', '')
df_cleaned_car_spec['EPA Classification'] = df_cleaned_car_spec['EPA Classification'].str.replace('-', '')

In [0]:
df_cleaned_car_spec['EPA Classification'] = df_cleaned_car_spec['EPA Classification'].apply(lambda x: 'na' if (x is None or float) else String(x).lower())

In [0]:
cols = ['Fuel Tank Capacity, Approx (gal)']
for col in cols:
  df_cleaned_car_spec[col] = df_cleaned_car_spec[col].apply(lambda v: tryconvert(v, float))
  print(df_cleaned_car_spec[col].isna().value_counts())

In [0]:
storeDf[ col + ' Float'] = storeDf[col].apply(lambda v: tryconvert(v, float))

In [0]:
def tryconvert(v, default):
  if v is None:
    return None
  try:
    return float(v)
  except ValueError:
    return None

In [0]:
list(df_cleaned_car_spec.columns.values)

In [0]:
df_cleaned_car_spec.dtypes

In [0]:
df_cleaned_car_spec['Body Style'] = df_cleaned_car_spec['Body Style'].apply(lambda v: tryconvert(v, str))

In [0]:
cols = ['Height, Overall (in)',
'Base Curb Weight (lbs)','Front Hip Room (in)','Front Leg Room (in)','Length, Overall (in)',
'Second Head Room (in)','Front Shoulder Room (in)',
'Front Head Room (in)','Second Leg Room (in)','Track Width, Front (in)','Track Width, Rear (in)','Second Shoulder Room (in)','Second Hip Room (in)'   ]
#for col in cols:
del df_cleaned_car_spec['Passenger Volume (ft³)']

In [0]:
transmission = []    
for values in df_cleaned_car_spec['SAE Net Horsepower @ RPM']:
  res = re.search("[0-9]* @", values)
  if res is None:
    transmission.append(None)
  if res is not None:
    transmission.append(res.group())

In [0]:
df_cleaned_car_spec['Horsepower'] = transmission

In [0]:
df_cleaned_car_spec['Horsepower'] = df_cleaned_car_spec['Horsepower'].str.replace('@', '')
df_cleaned_car_spec['Horsepower'] = df_cleaned_car_spec['Horsepower'].str.replace(' ', '')
df_cleaned_car_spec['Horsepower'].value_counts()

In [0]:
df_cleaned_car_spec['Secondary Make'] = df_cleaned_car_spec['Secondary Make'].apply(lambda v: tryconvert(v, str))

In [0]:
cols = [
 'Child Safety Rear Door Locks',
 'Daytime Running Lights',
 'Traction Control',
 'Rollover Protection Bars',
 'Fog Lamps',
 'Parking Aid',
 'Tire Pressure Monitor',
 'Back-Up Camera',
 'Stability Control'
]
for col in cols:
  
  df_cleaned_car_spec[col] = df_cleaned_car_spec[col].apply(lambda x: 1 if (x == 'Yes') else 0)
  print(df_cleaned_car_spec[col].value_counts())

In [0]:
df_cleaned_car_spec['Length, Overall (in)'] = df_cleaned_car_spec['Length, Overall (in)'].apply(lambda v: tryconvert(v, float))

In [0]:
# Corrosion Years                      object
# Drivetrain Miles/km                  object
# Drivetrain Years                     object
# Roadside Assistance Miles/km         object
# Roadside Assistance Years 

#labelEncoder = preprocessing.LabelEncoder()

col = 'Engine Fuel Type'
df_cleaned_car_spec[col] = df_cleaned_car_spec[col].apply(lambda x: 'na' if (x is None) else str(x).lower())
df_cleaned_car_spec[col] = df_cleaned_car_spec[col].str.replace(' ', '')
df_cleaned_car_spec[col] = df_cleaned_car_spec[col].str.replace(',', '')

In [0]:
col = 'Main Make'
labelEncoder = preprocessing.LabelEncoder()
value = list(df_cleaned_car_spec[col].values.astype(str))
labelEncoder.fit(value)
df_cleaned_car_spec[col] = labelEncoder.transform(df_cleaned_car_spec[col].astype(str))

In [0]:
df_cleaned_car_spec[col].value_counts()

In [0]:
breakMap = {'Pwr' : 0, '4-Wheel Disc' : 1, 'Power' : 0,'Pwr-Assisted' : 0,'Front Disc/Rear Drum' : 2, 'Hydraulic': 3,
'Pwr Regenerative' : 0, 'Pwr w/Hydroboost' : 0, '- TBD -' : 0,'Pwr-assisted' : 0, 'Pwr Assisted' : 0,
'Vacuum-assisted' : 4, 'pwr' : 0, 'Pwr Rack & Pinion' : 0}
df_cleaned_car_spec['Brake Type'] = df_cleaned_car_spec['Brake Type'].apply(lambda x: breakMap[x])

In [0]:

absMap = {'4wheel' : 0, 'nan' : 1, 'fourwheel' : 0,'4channel' : 2,'yes' : 0, 'rearwheel': 3,
'4wheel,4channel' : 4, 'tbd' : 1}
df_cleaned_car_spec['Brake ABS System'] = df_cleaned_car_spec['Brake ABS System'].apply(lambda x: absMap[x])

In [0]:
df_cleaned_car_spec['Make'].value_counts()

In [0]:
year = []    
for values in df_cleaned_car_spec['Car']:
  res = re.search("^[0-9]{4}", values)
  if res is None:
    year.append('2012')
  if res is not None:
    year.append(res.group())

In [0]:
df_cleaned_car_spec['Year'] = year

In [0]:
df_cleaned_car_spec['Year'] = df_cleaned_car_spec['Year'].astype(int)

In [0]:
df_cleaned_car_spec['Make'] = df_cleaned_car_spec['Car'].apply(lambda x: x.split('Specs')[0])

In [0]:
df_cleaned_car_spec[df_cleaned_car_spec['Car'].str.contains('2008 Ford F-150')]['Car']

In [0]:
df_cleaned_car_spec['Secondary Make'] = df_cleaned_car_spec['Make'].apply(lambda x: ' '.join(x.split(' ')[2:]))

In [549]:
round(df_cleaned_car_spec.isna().mean()*100,2).sort_values()

0
MRSP                                0.0
Stability Control                   0.0
Basic Miles/km                      0.0
Basic Years                         0.0
Corrosion Miles/km                  0.0
Corrosion Years                     0.0
Drivetrain Miles/km                 0.0
Drivetrain Years                    0.0
Roadside Assistance Miles/km        0.0
Roadside Assistance Years           0.0
City Mileage                        0.0
Highway Mileage                     0.0
Engine Fuel Type                    0.0
Engine Displacement                 0.0
Gears                               0.0
torque                              0.0
Horsepower                          0.0
Year                                0.0
Back-Up Camera                      0.0
Make                                0.0
Tire Pressure Monitor               0.0
Fog Lamps                           0.0
Drivetrain                          0.0
Passenger Capacity                  0.0
Passenger Doors                     0.

In [537]:
col = 'Passenger Volume (ft³)'
mode_v=df_cleaned_car_spec[col].mode()
print(mode_v)
mean_v=df_cleaned_car_spec[col].mean()
print(mean_v)
df_cleaned_car_spec[col].describe()

0    93.0
dtype: float64
100.52994549646819


count    14311.000000
mean       100.529945
std         22.466705
min         42.400000
25%         90.700000
50%         97.400000
75%        104.200000
max        188.400000
Name: Passenger Volume (ft³), dtype: float64

In [0]:
df_cleaned_car_spec['Passenger Volume (ft³)'].fillna(df_cleaned_car_spec['Passenger Volume (ft³)'].mode())

In [546]:
df_cleaned_car_spec[col].isna().value_counts()

False    14311
True      9811
Name: Passenger Volume (ft³), dtype: int64

In [0]:
df_cleaned_car_spec = df_cleaned_car_spec.dropna(axis=0, subset=['torque'])

In [551]:
df_cleaned_car_spec.shape

(24122, 41)

In [0]:
df_cleaned_car_spec.to_csv('cleaned.csv')